# Proyecto curso IA - Carlos Chabay

Esta notebook clasifica imágenes satelitales o de vuelos en dos categorías, con o sin edificaciones.
Recorre todas las imágenes ubicadas en la carpeta data y utilizando GPT-4o de OpenAI se genera una lista de diccionarios con el nombre de la imágen y True o False dependiendo si se observaron edificaciones.
Finalmente se genera un archivo result_edific.csv con el resultado de la clasificación para cada imágen.
Esta clasificación se utilizará para entrenar a un modelo YOLO para detectar edificaciones y mas adelante desarrollar un sistema en Geomática que compare edificaciones de la misma zona en distintos períodos.


In [10]:
import pandas as pd
import os
import base64
import requests
import time

def process_images(prompt, user_input, api_key, endpoint):

    # Itera sobre cada archivo .jpg en la carpeta data y devuelve una lista de diccionarios
    # con los nombres de las imágenes analizadas como clave y True o False dependiendo de si observó
    # edificaciones o no

    image_responses = {}
    for filename in os.listdir('data'):
        if filename.endswith(".jpg"):
            image_path = os.path.join('data/' + filename)

            # Encode the image in base64
            encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')

            # Set headers
            headers = {
                "Content-Type": "application/json",
                "api-key": api_key,
            }

            # Create payload for the request
            payload = {
                "messages": [
                    {
                        "role": "system",
                        "content": [
                            {
                                "type": "text",
                                "text": prompt
                            }
                        ]
                    },
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{encoded_image}"
                                }
                            },
                            {
                                "type": "text",
                                "text": user_input
                            }
                        ]
                    }
                ],
                "temperature": 0.7,
                "top_p": 0.95,
                "max_tokens": 800
            }

            # Send the request
            try:
                response = requests.post(endpoint, headers=headers, json=payload)
                response.raise_for_status()  # Will raise an HTTPError for bad responses
            except requests.RequestException as e:
                print(f"Error al procesar {filename}: {e}")
                continue

            # Extrae el contenido del mensaje
            response_json = response.json()
            message_content = response_json['choices'][0]['message']['content']

            # Guarda la respuesta de cada imagen en un diccionario con el nombre del archivo como clave
            image_responses[filename] = message_content

             # Esperar 20 segundos antes de hacer la próxima solicitud
            time.sleep(20)

    return image_responses

# Configura parámetros para la función que clasifica imágenes
api_key = ""
endpoint = "https://aoai-ine.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
prompt = "Eres un experto clasificador de imágenes satelitales. Debes indicar si una imagen contiene edificios."
user_input = '''"A partir de las siguiente imágen, devuelve True si hay edificios y False si no'''

# Invoca a la función que clasifica imágenes
responses = process_images(prompt, user_input, api_key, endpoint)

# Crea dataframe con resultado de clasificación
df = pd.DataFrame([[key, responses[key]] for key in responses.keys()], columns=['Imagen', 'Edific'])

print(df)

# Crea archivo result_edific.csv con el resultado de clasificación
df.to_csv('data/result_edific.csv', columns=['Imagen','Edific'], index=False)


                        Imagen Edific
0                  arboles.jpg  False
1  autopista_cancha_futbol.jpg   True
2     casas_piscina_estUTE.jpg   True
3            bosque_nativo.jpg  False
